Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab {-}
# Worksheet 2: Kinetics of Methylene-Blue RedOx {-}

In [ ]:
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import scipy.optimize

In [ ]:
# Load the google drive with your files 
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your collected data in .csv format
path='/content/drive/MyDrive/Colab Notebooks/Kinetics_Data/'

In [ ]:
# As a preliminary step, you need to create a .csv file with the important 
# information on each experiment and the corresponding filename.
# You can use the template provided by the instructors, but make sure check
# your experimental data and filenames
concentration_file='concentrations.csv'

In [ ]:
# The following commands will read all the .csv files and will create a 
# DataFrame with all the absorbance curves for the different experiments
#
concentrations=pd.read_csv(path+concentration_file)
maxtime = 0.
for index,row in concentrations.iterrows() : 
    data=pd.read_csv(path+row['Filename'],header=1,usecols=(0,1))
    data=data.apply(pd.to_numeric,errors='coerce').dropna()
    time = data.loc[len(data.index)-1,'Time (sec)']
    if time > maxtime : maxtime = time
full_data=pd.DataFrame({'time':np.arange(0.25,maxtime,0.5)})
for index,row in concentrations.iterrows() : 
    data=pd.read_csv(path+row['Filename'],header=1,usecols=(0,1))
    data=data.apply(pd.to_numeric,errors='coerce').dropna()
    full_data['abs'+row['Set']+str(row['Run'])]=data['Abs']

## Task 1: Order of the Reaction with Respect to Methylene Blue {-}

* Fit the absorbance decay curve for one of the experiments in Set A with three alternative models, each corresponding to a different kinetic order with respect to methylene blue. From the three plots verifies that the model that best describes the kinetics is the first order one. In fitting the three models, you will realize that the very first part of your experiment does not fit well with any of the models. This is because the transient regimes that occurs at the beginning of the experiment as you mix your reactants has a complicated behavior that does involve mass transport in the liquid and does not follow any of the steady-state kinetic models that you have studied. As part of this task, adjust the transient time until at least one of the fits appears reasonable. Report this transient time with your comments and keep it in mind for the later tasks.

YOUR COMMENTS HERE

In [ ]:
def linear(t, a, k, o):
    """ 
    Function that returns a linear decaying function that saturates to an offset
    f(t) = o + a - k * t (for t < a/k) or o (for t > a/k)

    input variables
    t: input value (units of time)
    a: amplitude (units of absorbance or concentration)
    k: decay rate (units of 1/time)
    o: offset (units of absorbance or concentration)
    """
    filter = t < a/k
    return o + filter * (a - k * t)

def exponential(t, a, k, o):
    """ 
    Function that returns an exponentially decaying function plus offset
    f(t) = o + a*e^(-k*t)

    input variables
    t: input value (units of time)
    a: amplitude (units of absorbance or concentration)
    k: decay rate (units of 1/time)
    o: offset (units of absorbance or concentration)
    """
    return a * np.exp(-k*t) + o

def inverse(t, a, k, o):
    """ 
    Function that returns a inverse decaying function plus offset
    f(t) = o + 1/(1/a + 2*k*t) 

    input variables
    t: input value (units of time)
    a: amplitude (units of absorbance or concentration)
    k: decay rate (units of 1/time)
    o: offset (units of absorbance or concentration)
    """
    return o + 1/(1/a+2*k*t)

In [ ]:
trans_time = 50 # transient time
end_time=400 # final collection time
dataset = 'absA1'
x,y=np.split(full_data.query('time > {} and time < {}'.format(trans_time,end_time))[['time',dataset]].dropna().values,2,1)
plt.plot(x,y)
#
# order 0: y(t) = y0 - k * t + offset
#
p0 = (0.45, 0.002, 0.05)
params, cv = scipy.optimize.curve_fit(linear, x.flatten(), y.flatten(), p0)
plt.plot(x,linear(x,params[0],params[1],params[2]))
#
# order 1: y(t) = y0 * exp(-k*t) + offset
#
p0 = (0.5, 1/85., 0.05)
params, cv = scipy.optimize.curve_fit(exponential, x.flatten(), y.flatten(), p0)
plt.plot(x,exponential(x,params[0],params[1],params[2]))
#
# order 2: y(t) = 1 / ( 1/y0 + k*t ) + offset
#
p0 = (0.5, 1/85., 0.05)
params, cv = scipy.optimize.curve_fit(inverse, x.flatten(), y.flatten(), p0)
plt.plot(x,inverse(x,params[0],params[1],params[2]))
#
plt.show()

* Describe the model that best fits your experiment, e.g. qualitatively and/or by comparing the $R^2$ coefficients of the different fits. Remember that the coefficient of determination can be computed as $R^2=1-RSS/TSS$, where $RSS$ is the residual sum of squares while $TSS$ is the total sum of squares. Add any additional comment or explanation.

YOUR COMMENTS HERE

* Knowing that methylene blue is the only chemical in the reaction that absorbs at around 665 nm, provide a plausible explanation of the offset observed in the decays of methylene blue.

YOUR COMMENTS HERE

## Task 2: Rate Constant in Flooding Conditions {-}

The first set of experiments (Set A) was performed under flooding conditions, i.e. with a much larger pH and concentration of ascorbic acid, [AA], than the concentration of methylene blue, [MB]. Under these conditions, the kinetics is first order in [MB] and the fit using an expontential function plus offset should provide a good estimate of the effective rate constant.
* Fit the absorbance decay curves for the experiments. Plot the decay curves together with the fitted curves. Make sure the plots look good and have all the necessary information in them. 
* Do the effective rate constants that you determine depend on precisely konwing the starting concentration of methylene blue? Why or why not?

In [ ]:
trans_time = 50 # transient time
end_time = 400 # final collection time
p0 = (0.5, 1/85., 0.05)
for index,row in concentrations[concentrations['Set']=='A'].iterrows() : 
    x,y=np.split(full_data[full_data['time']>80][['time','absA'+str(row['Run'])]].dropna().values,2,1)
    params, cv = scipy.optimize.curve_fit(exponential, x.flatten(), y.flatten(), p0)
    concentrations.loc[index,'y0']=params[0]
    concentrations.loc[index,'y0_sigma']=np.sqrt(cv[0,0])
    concentrations.loc[index,'k']=params[1]
    concentrations.loc[index,'k_sigma']=np.sqrt(cv[1,1])
    concentrations.loc[index,'offset']=params[2]
    concentrations.loc[index,'offset_sigma']=np.sqrt(cv[2,2])
    full_data['fitA'+str(row['Run'])]=exponential(full_data['time'].values,params[0],params[1],params[2]) 
plt.semilogy(full_data['time'],full_data['absA1'])
plt.semilogy(full_data['time'],full_data['fitA1'])
plt.semilogy(full_data['time'],full_data['absA2'])
plt.semilogy(full_data['time'],full_data['fitA2'])
plt.semilogy(full_data['time'],full_data['absA3'])
plt.semilogy(full_data['time'],full_data['fitA3'])
plt.semilogy(full_data['time'],full_data['absA4'])
plt.semilogy(full_data['time'],full_data['fitA4'])
plt.show()

YOUR COMMENTS HERE

* Report your best estimate for the effective rate constant and try to address these points in your discussion of the results:
  * Do your estimates in the four runs have the same accuracy? 
  * If not, which one is less accurate? Why? 

In [ ]:
print("The best estimate of the effective decay rate for MB redox under flooding conditions is {} 1/s".format(0.))
print("This estimate was obtained under the following conditions: [AA] = {} M and [HCl] = {} M".format(0.,0.))
print("The error on the estimated decay rate is {} 1/s".format(0.0))

YOUR COMMENTS HERE

## Task 3: Reaction Order with Respect to Ascorbic Acid {-}

In the second set of experiments (Set B) you kept $[MB]$ fixed and varied the concentration of ascorbic acid. However, your UV/Vis apparatus is still tracking the behavior of the methylene blue. The effective decay rate of MB will be given by $k_{eff}^B=k[MB]^m[AA]^n[HCl]^p$, where the $[MB]$ and $[HCl]$ terms will be the same in the four runs. By fitting the effective decay rate as a function of $[AA]$ we can determine the reaction order with respect to the second redox compound. 

* Fit the exponential decays of the different runs and extract the effective decay rates. 
* Plot the behavior of $\log(k_{eff}^B)$ vs. $\log([AA])$, the slope of the plot should provide an estimate for the order $n$ of the reaction vs. [AA]. 
  * Is the order of the reaction the one you expected? Why or why not?

In [ ]:
trans_time = 0 # transient time
end_time = 400 # final collection time
p0 = (0.5, 1/85., 0.05)
for index,row in concentrations[concentrations['Set']=='B'].iterrows() : 
    x,y=np.split(full_data[full_data['time']>80][['time','absB'+str(row['Run'])]].dropna().values,2,1)
    params, cv = scipy.optimize.curve_fit(exponential, x.flatten(), y.flatten(), p0)
    concentrations.loc[index,'y0']=params[0]
    concentrations.loc[index,'y0_sigma']=np.sqrt(cv[0,0])
    concentrations.loc[index,'k']=params[1]
    concentrations.loc[index,'k_sigma']=np.sqrt(cv[1,1])
    concentrations.loc[index,'offset']=params[2]
    concentrations.loc[index,'offset_sigma']=np.sqrt(cv[2,2])
    full_data['fitB'+str(row['Run'])]=exponential(full_data['time'].values,params[0],params[1],params[2])

In [ ]:
x=np.log(concentrations[concentrations['Set']=='B']['AA'].values[:-1].reshape(-1,1))
y=np.log(concentrations[concentrations['Set']=='B']['k'].values[:-1])
aa_order=LinearRegression()
aa_order.fit(x,y)
print("The order of the reaction with respect to [AA] is {}".format(aa_order.coef_[0]))
plt.plot(x,y,'o')
plt.plot(x,aa_order.predict(x))
plt.show()

YOUR COMMENTS HERE

## Task 4: Reaction Order with Respect to pH {-}

In the second set of experiments (Set B) you kept $[MB]$ fixed and varied the concentration of HCl, i.e. you changed the pH of the solution. However, your UV/Vis apparatus is still tracking the behavior of the methylene blue. The effective decay rate of MB will be given by $k_{eff}^C=k[MB]^m[AA]^n[HCl]^p$, where the $[MB]$ and $[AA]$ terms will be the same in the four runs. By fitting the effective decay rate as a function of $[HCl]$ or pH we can determine the reaction order with respect to a potential component of the redox reaction. 

* Fit the exponential decays of the different runs and extract the effective decay rates. 
* Plot the behavior of $\log(k_{eff}^C)$ vs. $\log([HCl])$, the slope of the plot should provide an estimate for the order of the reaction vs. [HCl].
  * Is there a clear order of the reaction with respect to [HCl]? What could be the role of pH?

In [ ]:
trans_time = 0 # transient time
end_time = 400 # final collection time
p0 = (0.5, 1/85., 0.05)
for index,row in concentrations[concentrations['Set']=='C'].iterrows() : 
    x,y=np.split(full_data[full_data['time']>80][['time','absC'+str(row['Run'])]].dropna().values,2,1)
    params, cv = scipy.optimize.curve_fit(exponential, x.flatten(), y.flatten(), p0)
    concentrations.loc[index,'y0']=params[0]
    concentrations.loc[index,'y0_sigma']=np.sqrt(cv[0,0])
    concentrations.loc[index,'k']=params[1]
    concentrations.loc[index,'k_sigma']=np.sqrt(cv[1,1])
    concentrations.loc[index,'offset']=params[2]
    concentrations.loc[index,'offset_sigma']=np.sqrt(cv[2,2])
    full_data['fitC'+str(row['Run'])]=exponential(full_data['time'].values,params[0],params[1],params[2])

In [ ]:
x=np.log(concentrations[concentrations['Set']=='C']['HCl'].values[:-1].reshape(-1,1))
y=np.log(concentrations[concentrations['Set']=='C']['k'].values[:-1])
hcl_order=LinearRegression()
hcl_order.fit(x,y)
print("The order of the reaction with respect to [HCl] is {}".format(hcl_order.coef_[0]))
plt.plot(x,y,'o')
plt.plot(x,hcl_order.predict(x))
plt.show()

YOUR COMMENTS HERE

In [ ]:
# This cell is used to allow Google Colab to install the tools to convert the notebook to a pdf file
# Un-comment the following lines when you are ready to export the pdf 
#!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
#!pip install pypandoc

In [ ]:
# Use this command to convert the finished worksheet into a pdf 
# NOTE : you may want to change the path of the file, if you are working in a different folder of the Google Drive
#!jupyter nbconvert --no-input --to PDF "/content/drive/MyDrive/Colab Notebooks/Kinetics_Worksheet.ipynb"